# Lab 2: Data Preparation — UCI Bank Marketing

In Lab 1 we profiled the Bank Marketing campaign data and flagged issues such as placeholder `unknown` tokens, skewed numerics (`balance`, `campaign`, `pdays`), and the leakage risk of the `duration` field. This lab builds a reproducible preprocessing workflow that cleans, encodes, and scales the dataset for machine-learning models.

## What we will address

- Treat missing information (MCAR/MAR/MNAR) via column drops, row filtering, and imputations (median, mode, KNN).
- Encode categorical features (ordinal vs nominal) and ensure the target is numeric.
- Handle outliers by flagging, transforming, and trimming extreme records.
- Engineer extra features capturing campaign behaviour and calendar signals.
- Rescale/normalise numeric features with `RobustScaler`, `StandardScaler`, `MinMaxScaler`, and `Normalizer`.

## A. Import Python Modules and Dataset

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, Normalizer

%matplotlib inline

plt.style.use("seaborn-v0_8")
pd.options.display.float_format = "{:.2f}".format

df = pd.read_csv("data/bank.csv", sep=";")
df["row_id"] = np.arange(len(df))
df_raw = df.copy()  # keep pristine copy for derived features
df.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,row_id
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no,4516
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no,4517
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no,4518
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no,4519
4520,44,entrepreneur,single,tertiary,no,1136,yes,yes,cellular,3,apr,345,2,249,7,other,no,4520


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
 17  row_id     4521 non-null   int64 
dtypes: int64(8), object(10)
memory usage: 635.9+ KB


## B. Process and Encode the Categorical Features

### Drop non-informative / leakage-prone columns

`duration` is measured after the telemarketing outcome and leaks future information, so we remove it upfront.

In [89]:
df.drop(columns=["duration"], inplace=True)

In [90]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y,row_id
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,1,-1,0,unknown,no,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,1,339,4,failure,no,1
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,1,330,1,failure,no,2
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,4,-1,0,unknown,no,3
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,1,-1,0,unknown,no,4


### Encode ordinal attributes

Education level and campaign month have an intrinsic order. We map them to ordered integers while keeping missing entries for later imputation.

In [91]:
education_mapper = {"primary": 0, "secondary": 1, "tertiary": 2}
month_mapper = {
    "jan": 1, "feb": 2, "mar": 3, "apr": 4, "may": 5, "jun": 6,
    "jul": 7, "aug": 8, "sep": 9, "oct": 10, "nov": 11, "dec": 12,
}

df["education"] = df["education"].map(education_mapper)
df["month"] = df["month"].map(month_mapper)

df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y,row_id
0,30,unemployed,married,0.00,no,1787,no,no,cellular,19,10,1,-1,0,unknown,no,0
1,33,services,married,1.00,no,4789,yes,yes,cellular,11,5,1,339,4,failure,no,1
2,35,management,single,2.00,no,1350,yes,no,cellular,16,4,1,330,1,failure,no,2
3,30,management,married,2.00,no,1476,yes,yes,unknown,3,6,4,-1,0,unknown,no,3
4,59,blue-collar,married,1.00,no,0,yes,no,unknown,5,5,1,-1,0,unknown,no,4


### Binary-encode the target

We convert the subscription outcome to 0/1 for ML compatibility.

In [92]:
df["y"] = df["y"].map({"no": 0, "yes": 1})

df.head(20)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y,row_id
0,30,unemployed,married,0.00,no,1787,no,no,cellular,19,10,1,-1,0,unknown,0,0
1,33,services,married,1.00,no,4789,yes,yes,cellular,11,5,1,339,4,failure,0,1
2,35,management,single,2.00,no,1350,yes,no,cellular,16,4,1,330,1,failure,0,2
3,30,management,married,2.00,no,1476,yes,yes,unknown,3,6,4,-1,0,unknown,0,3
4,59,blue-collar,married,1.00,no,0,yes,no,unknown,5,5,1,-1,0,unknown,0,4
5,35,management,single,2.00,no,747,no,no,cellular,23,2,2,176,3,failure,0,5
6,36,self-employed,married,2.00,no,307,yes,no,cellular,14,5,1,330,2,other,0,6
7,39,technician,married,1.00,no,147,yes,no,cellular,6,5,2,-1,0,unknown,0,7
8,41,entrepreneur,married,2.00,no,221,yes,no,unknown,14,5,2,-1,0,unknown,0,8
9,43,services,married,0.00,no,-88,yes,yes,cellular,17,4,1,147,2,failure,0,9


### One-hot encode nominal attributes

Nominal categoricals are expanded into binary indicators so that models receive purely numeric inputs.

In [93]:
nominal_cols = ["job", "marital", "default", "housing", "loan", "contact"]
df = pd.get_dummies(df, columns=nominal_cols, drop_first=True)

In [94]:
df.head(20)

,age,education,balance,day,month,campaign,pdays,previous,poutcome,y,...,job_technician,job_unemployed,job_unknown,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,contact_unknown
0,30,0.00,1787,19,10,1,-1,0,unknown,0,...,False,True,False,True,False,False,False,False,False,False
1,33,1.00,4789,11,5,1,339,4,failure,0,...,False,False,False,True,False,False,True,True,False,False
2,35,2.00,1350,16,4,1,330,1,failure,0,...,False,False,False,False,True,False,True,False,False,False
3,30,2.00,1476,3,6,4,-1,0,unknown,0,...,False,False,False,True,False,False,True,True,False,True
4,59,1.00,0,5,5,1,-1,0,unknown,0,...,False,False,False,True,False,False,True,False,False,True
5,35,2.00,747,23,2,2,176,3,failure,0,...,False,False,False,False,True,False,False,False,False,False
6,36,2.00,307,14,5,1,330,2,other,0,...,False,False,False,True,False,False,True,False,False,False
7,39,1.00,147,6,5,2,-1,0,unknown,0,...,True,False,False,True,False,False,True,False,False,False
8,41,2.00,221,14,5,2,-1,0,unknown,0,...,False,False,False,True,False,False,True,False,False,True
9,43,0.00,-88,17,4,1,147,2,failure,0,...,False,False,False,True,False,False,True,True,False,False


## C. Treatment of Missing Values

### Identify missing values

We replace sentinel values (`"unknown"`, `pdays = -1`) with proper `NaN` so pandas can track them, then summarise the missingness profile.

In [95]:
object_cols = df.select_dtypes(include="object").columns
df[object_cols] = df[object_cols].replace("unknown", np.nan)
df["pdays"] = df["pdays"].replace(-1, np.nan)

missing_summary = (
    df.isna()
      .sum()
      .to_frame(name="missing_count")
)
missing_summary["missing_pct"] = (missing_summary["missing_count"] / len(df) * 100).round(2)
missing_summary.sort_values("missing_pct", ascending=False)

,missing_count,missing_pct
pdays,3705,81.95
poutcome,3705,81.95
education,187,4.14
age,0,0.00
job_self-employed,0,0.00
contact_telephone,0,0.00
loan_yes,0,0.00
housing_yes,0,0.00
default_yes,0,0.00
marital_single,0,0.00


In [96]:
df.head(20)

,age,education,balance,day,month,campaign,pdays,previous,poutcome,y,...,job_technician,job_unemployed,job_unknown,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,contact_unknown
0,30,0.00,1787,19,10,1,NaN,0,NaN,0,...,False,True,False,True,False,False,False,False,False,False
1,33,1.00,4789,11,5,1,339.00,4,failure,0,...,False,False,False,True,False,False,True,True,False,False
2,35,2.00,1350,16,4,1,330.00,1,failure,0,...,False,False,False,False,True,False,True,False,False,False
3,30,2.00,1476,3,6,4,NaN,0,NaN,0,...,False,False,False,True,False,False,True,True,False,True
4,59,1.00,0,5,5,1,NaN,0,NaN,0,...,False,False,False,True,False,False,True,False,False,True
5,35,2.00,747,23,2,2,176.00,3,failure,0,...,False,False,False,False,True,False,False,False,False,False
6,36,2.00,307,14,5,1,330.00,2,other,0,...,False,False,False,True,False,False,True,False,False,False
7,39,1.00,147,6,5,2,NaN,0,NaN,0,...,True,False,False,True,False,False,True,False,False,False
8,41,2.00,221,14,5,2,NaN,0,NaN,0,...,False,False,False,True,False,False,True,False,False,True
9,43,0.00,-88,17,4,1,147.00,2,failure,0,...,False,False,False,True,False,False,True,True,False,False


### Missingness types (MCAR / MAR / MNAR)

- **MNAR (`pdays`, `poutcome`)**: missing because the client was never re-contacted; absence encodes information about previous campaigns.
- **MAR (`contact`)**: channel is absent mainly when calls were routed differently; depends on other features such as job type.
- **Approximately MCAR (`education`, `job`)**: sparse blanks likely stem from data entry noise and are treated as random.

### Engineer indicators before dropping high-missing columns

We keep the signal from MNAR fields by deriving explicit indicators, then drop columns exceeding the 30% threshold (`poutcome`, `pdays`).

In [97]:
df["pdays_recency"] = df["pdays"]
df["pdays_missing"] = df["pdays_recency"].isna().astype(int)
df["contacted_before"] = df["pdays_recency"].notna().astype(int)
df["recent_contact_90d"] = df["pdays_recency"].le(90).fillna(False).astype(int)

df["poutcome_missing"] = df["poutcome"].isna().astype(int)
df["poutcome_success_flag"] = (df["poutcome"] == "success").astype(int)
df["poutcome_failure_flag"] = (df["poutcome"] == "failure").astype(int)

high_missing_cols = missing_summary[missing_summary["missing_pct"] > 30].index.tolist()
if "pdays" in high_missing_cols:
    high_missing_cols.remove("pdays")

df.drop(columns=high_missing_cols, inplace=True)
df.drop(columns=["pdays"], inplace=True)

### Drop rows with excessive missingness

Rows with more than 50% missing fields offer little value and are removed (the dataset currently has none, but the guard is kept for reproducibility).

In [98]:
row_missing_ratio = df.isna().mean(axis=1)
rows_to_drop = df[row_missing_ratio > 0.5].index
df.drop(index=rows_to_drop, inplace=True)
print(f"Rows dropped: {len(rows_to_drop)}")

Rows dropped: 0


### Impute remaining categorical and ordinal fields

We fill categorical blanks with the mode and ordinal numerics with the median. `contact_missing` preserves whether the channel was recorded.

In [ ]:
df["contact_missing"] = df["contact"].isna().astype(int)

categorical_mode_cols = ["job", "marital", "default", "housing", "loan", "contact"]
cat_imputer = SimpleImputer(strategy="most_frequent")
df[categorical_mode_cols] = cat_imputer.fit_transform(df[categorical_mode_cols])

numeric_median_cols = ["education", "month"]
num_imputer = SimpleImputer(strategy="median")
df[numeric_median_cols] = num_imputer.fit_transform(df[numeric_median_cols])

### KNN imputation for `pdays_recency`

We approximate missing recency values using nearby samples in the `campaign`, `previous`, and `age` space.

In [13]:
knn_features = ["pdays_recency", "campaign", "previous", "age"]
knn_imputer = KNNImputer(n_neighbors=5, weights="distance")
df[knn_features] = knn_imputer.fit_transform(df[knn_features])

### Missing-value audit after imputations

In [14]:
df.isna().sum()

age                      0
job                      0
marital                  0
education                0
default                  0
balance                  0
housing                  0
loan                     0
contact                  0
day                      0
month                    0
campaign                 0
previous                 0
y                        0
row_id                   0
pdays_recency            0
pdays_missing            0
contacted_before         0
recent_contact_90d       0
poutcome_missing         0
poutcome_success_flag    0
poutcome_failure_flag    0
contact_missing          0
dtype: int64

## D. Rescaling Features

Robust, standard, and min–max scalers reduce scale disparities and mitigate the influence of outliers before modelling.

In [17]:
robust_scaler = RobustScaler()
df[["balance", "campaign"]] = robust_scaler.fit_transform(df[["balance", "campaign"]])

standard_scaler = StandardScaler()
df[["age"]] = standard_scaler.fit_transform(df[["age"]])

minmax_scaler = MinMaxScaler()
df[["day"]] = minmax_scaler.fit_transform(df[["day"]])

In [18]:
df.head(20)

,age,education,balance,day,month,campaign,previous,y,row_id,pdays_recency,...,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone
0,-1.06,0.00,0.95,0.60,10.00,-0.50,0.00,0,0,168.43,...,False,False,False,True,True,False,False,False,False,False
1,-0.77,1.00,3.08,0.33,5.00,-0.50,4.00,0,1,339.00,...,True,False,False,False,True,False,False,True,True,False
2,-0.58,2.00,0.64,0.50,4.00,-0.50,1.00,0,2,330.00,...,False,False,False,False,False,True,False,True,False,False
3,-1.06,2.00,0.73,0.07,6.00,1.00,0.00,0,3,290.76,...,False,False,False,False,True,False,False,True,True,False
4,1.69,1.00,-0.31,0.13,5.00,-0.50,0.00,0,4,149.46,...,False,False,False,False,True,False,False,True,False,False
5,-0.58,2.00,0.21,0.73,2.00,0.00,3.00,0,5,176.00,...,False,False,False,False,False,True,False,False,False,False
6,-0.49,2.00,-0.10,0.43,5.00,-0.50,2.00,0,6,330.00,...,False,False,False,False,True,False,False,True,False,False
7,-0.21,1.00,-0.21,0.17,5.00,0.00,0.00,0,7,306.74,...,False,False,True,False,True,False,False,True,False,False
8,-0.02,2.00,-0.16,0.43,5.00,0.00,0.00,0,8,284.66,...,False,False,False,False,True,False,False,True,False,False
9,0.17,0.00,-0.38,0.53,4.00,-0.50,2.00,0,9,147.00,...,True,False,False,False,True,False,False,True,True,False


## E. Treatment of Outliers

We flag distribution extremes, drop clearly unrepresentative rows, and apply monotonic transforms to reduce skewness.

1) The IQR outlier function:

In [ ]:
def iqr_outlier_mask(series):
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return lower, upper, ((series < lower) | (series > upper)).astype(int)

balance_lower, balance_upper, balance_mask = iqr_outlier_mask(df["balance"])
campaign_lower, campaign_upper, campaign_mask = iqr_outlier_mask(df["campaign"])

df["balance_outlier_flag"] = balance_mask
df["campaign_outlier_flag"] = campaign_mask

In [ ]:
df.head(20)

,age,education,balance,day,month,campaign,previous,y,row_id,pdays_recency,...,job_technician,job_unemployed,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,balance_outlier_flag,campaign_outlier_flag
0,-1.06,0.00,0.95,0.60,10.00,-0.50,0.00,0,0,168.43,...,False,True,True,False,False,False,False,False,0,0
1,-0.77,1.00,3.08,0.33,5.00,-0.50,4.00,0,1,339.00,...,False,False,True,False,False,True,True,False,1,0
2,-0.58,2.00,0.64,0.50,4.00,-0.50,1.00,0,2,330.00,...,False,False,False,True,False,True,False,False,0,0
3,-1.06,2.00,0.73,0.07,6.00,1.00,0.00,0,3,290.76,...,False,False,True,False,False,True,True,False,0,0
4,1.69,1.00,-0.31,0.13,5.00,-0.50,0.00,0,4,149.46,...,False,False,True,False,False,True,False,False,0,0
5,-0.58,2.00,0.21,0.73,2.00,0.00,3.00,0,5,176.00,...,False,False,False,True,False,False,False,False,0,0
6,-0.49,2.00,-0.10,0.43,5.00,-0.50,2.00,0,6,330.00,...,False,False,True,False,False,True,False,False,0,0
7,-0.21,1.00,-0.21,0.17,5.00,0.00,0.00,0,7,306.74,...,True,False,True,False,False,True,False,False,0,0
8,-0.02,2.00,-0.16,0.43,5.00,0.00,0.00,0,8,284.66,...,False,False,True,False,False,True,False,False,0,0
9,0.17,0.00,-0.38,0.53,4.00,-0.50,2.00,0,9,147.00,...,False,False,True,False,False,True,True,False,0,0


2) Trimming extreme rows using a percentile cutoff

In [ ]:
campaign_cutoff = np.quantile(df["campaign"], 0.99)
rows_before = len(df)
df = df[df["campaign"] <= campaign_cutoff].reset_index(drop=True)
rows_removed = rows_before - len(df)
print(f"Rows removed due to extreme campaign counts (> {campaign_cutoff:.2f}): {rows_removed}")

Rows removed due to extreme campaign counts (> 7.00): 43


In [ ]:
df.head(20)

,age,education,balance,day,month,campaign,previous,y,row_id,pdays_recency,...,job_technician,job_unemployed,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,balance_outlier_flag,campaign_outlier_flag
0,-1.06,0.00,0.95,0.60,10.00,-0.50,0.00,0,0,168.43,...,False,True,True,False,False,False,False,False,0,0
1,-0.77,1.00,3.08,0.33,5.00,-0.50,4.00,0,1,339.00,...,False,False,True,False,False,True,True,False,1,0
2,-0.58,2.00,0.64,0.50,4.00,-0.50,1.00,0,2,330.00,...,False,False,False,True,False,True,False,False,0,0
3,-1.06,2.00,0.73,0.07,6.00,1.00,0.00,0,3,290.76,...,False,False,True,False,False,True,True,False,0,0
4,1.69,1.00,-0.31,0.13,5.00,-0.50,0.00,0,4,149.46,...,False,False,True,False,False,True,False,False,0,0
5,-0.58,2.00,0.21,0.73,2.00,0.00,3.00,0,5,176.00,...,False,False,False,True,False,False,False,False,0,0
6,-0.49,2.00,-0.10,0.43,5.00,-0.50,2.00,0,6,330.00,...,False,False,True,False,False,True,False,False,0,0
7,-0.21,1.00,-0.21,0.17,5.00,0.00,0.00,0,7,306.74,...,True,False,True,False,False,True,False,False,0,0
8,-0.02,2.00,-0.16,0.43,5.00,0.00,0.00,0,8,284.66,...,False,False,True,False,False,True,False,False,0,0
9,0.17,0.00,-0.38,0.53,4.00,-0.50,2.00,0,9,147.00,...,False,False,True,False,False,True,True,False,0,0


3) Monotonic (log) transforms to reduce skewness

In [ ]:
balance_shift = np.abs(df["balance"].min()) + 1
df["balance"] = np.log(df["balance"] + balance_shift)

campaign_shift = np.abs(df["campaign"].min()) + 1
df["campaign"] = np.log(df["campaign"] + campaign_shift)

df["previous"] = np.log1p(df["previous"])
df["pdays_recency"] = np.log1p(np.maximum(df["pdays_recency"], 0))

In [ ]:
df.head(20)

,age,education,balance,day,month,campaign,previous,y,row_id,pdays_recency,...,job_technician,job_unemployed,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,balance_outlier_flag,campaign_outlier_flag
0,-1.06,0.00,1.53,0.60,10.00,0.00,0.00,0,0,5.13,...,False,True,True,False,False,False,False,False,0,0
1,-0.77,1.00,1.91,0.33,5.00,0.00,1.61,0,1,5.83,...,False,False,True,False,False,True,True,False,1,0
2,-0.58,2.00,1.46,0.50,4.00,0.00,0.69,0,2,5.80,...,False,False,False,True,False,True,False,False,0,0
3,-1.06,2.00,1.48,0.07,6.00,0.92,0.00,0,3,5.68,...,False,False,True,False,False,True,True,False,0,0
4,1.69,1.00,1.21,0.13,5.00,0.00,0.00,0,4,5.01,...,False,False,True,False,False,True,False,False,0,0
5,-0.58,2.00,1.36,0.73,2.00,0.41,1.39,0,5,5.18,...,False,False,False,True,False,False,False,False,0,0
6,-0.49,2.00,1.27,0.43,5.00,0.00,1.10,0,6,5.80,...,False,False,True,False,False,True,False,False,0,0
7,-0.21,1.00,1.24,0.17,5.00,0.41,0.00,0,7,5.73,...,True,False,True,False,False,True,False,False,0,0
8,-0.02,2.00,1.25,0.43,5.00,0.41,0.00,0,8,5.65,...,False,False,True,False,False,True,False,False,0,0
9,0.17,0.00,1.19,0.53,4.00,0.00,1.10,0,9,5.00,...,False,False,True,False,False,True,True,False,0,0


### Additional scaling after outlier treatment

4) Flags, re-scaling after transforms, and Normalizer

We re-standardise the transformed features and add row-wise normalised contact intensity signals.

In [ ]:
post_log_scaler = StandardScaler()
df[["previous", "pdays_recency"]] = post_log_scaler.fit_transform(df[["previous", "pdays_recency"]])

normalizer = Normalizer()
intensity_matrix = normalizer.fit_transform(df[["campaign", "previous", "pdays_recency"]])
df[["campaign_unit", "previous_unit", "pdays_unit"]] = intensity_matrix

In [ ]:
df.head(20)

,age,education,balance,day,month,campaign,previous,y,row_id,pdays_recency,...,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,balance_outlier_flag,campaign_outlier_flag,campaign_unit,previous_unit,pdays_unit
0,-1.06,0.00,1.53,0.60,10.00,0.00,-0.43,0,0,-0.74,...,False,False,False,False,False,0,0,0.00,-0.50,-0.87
1,-0.77,1.00,1.91,0.33,5.00,0.00,2.65,0,1,1.03,...,False,False,True,True,False,1,0,0.00,0.93,0.36
2,-0.58,2.00,1.46,0.50,4.00,0.00,0.90,0,2,0.96,...,True,False,True,False,False,0,0,0.00,0.68,0.73
3,-1.06,2.00,1.48,0.07,6.00,0.92,-0.43,0,3,0.64,...,False,False,True,True,False,0,0,0.77,-0.36,0.53
4,1.69,1.00,1.21,0.13,5.00,0.00,-0.43,0,4,-1.04,...,False,False,True,False,False,0,0,0.00,-0.38,-0.93
5,-0.58,2.00,1.36,0.73,2.00,0.41,2.22,0,5,-0.63,...,True,False,False,False,False,0,0,0.17,0.95,-0.27
6,-0.49,2.00,1.27,0.43,5.00,0.00,1.67,0,6,0.96,...,False,False,True,False,False,0,0,0.00,0.87,0.50
7,-0.21,1.00,1.24,0.17,5.00,0.41,-0.43,0,7,0.77,...,False,False,True,False,False,0,0,0.42,-0.44,0.80
8,-0.02,2.00,1.25,0.43,5.00,0.41,-0.43,0,8,0.58,...,False,False,True,False,False,0,0,0.49,-0.51,0.70
9,0.17,0.00,1.19,0.53,4.00,0.00,1.67,0,9,-1.09,...,False,False,True,True,False,0,0,0.00,0.84,-0.54


## F. Creation of Derived Features

In this section, we create new features that capture important relationships and patterns in our data. We create four main types of derived features:

1. **Balance Per Call** 
   - Calculation: `balance / campaign`
   - Purpose: Measures the average account balance per campaign contact
  

2. **Contact Intensity** 
   - Calculation: `campaign / (previous + 1)`
   - Purpose: Measures how intensively the current campaign contacted each client
   
3. **Cyclic Month Encoding**
   - Calculations:
     * `month_sin = sin(2π × month / 12)`
     * `month_cos = cos(2π × month / 12)`
   - Purpose: Transforms month numbers into continuous cyclic features
 
     ```

4. **Data Handling Details**
   - Use raw data (`df_raw_indexed`) to ensure calculations use original values
   - Handle missing values with median imputation
   - Maintain proper alignment using row_ids
   - Apply appropriate scaling to each derived feature



In [ ]:
# Visualize cyclic month encoding
import matplotlib.pyplot as plt
import numpy as np

months = np.arange(1, 13)
month_sin = np.sin(2 * np.pi * months / 12)
month_cos = np.cos(2 * np.pi * months / 12)

plt.figure(figsize=(10, 5))

# Plot the sine and cosine transformations
plt.subplot(121)
plt.plot(months, month_sin, 'b-', label='sin(month)')
plt.plot(months, month_cos, 'r-', label='cos(month)')
plt.xlabel('Month')
plt.ylabel('Value')
plt.title('Sine and Cosine Transformations')
plt.grid(True)
plt.legend()

# Plot the circular representation
plt.subplot(122)
plt.plot(month_cos, month_sin, 'g-')
plt.scatter(month_cos, month_sin, c=months, cmap='viridis')
for i, month in enumerate(months):
    plt.annotate(f'Month {month}', (month_cos[i], month_sin[i]))
plt.xlabel('Cosine Component')
plt.ylabel('Sine Component')
plt.title('Circular Month Representation')
plt.grid(True)
plt.axis('equal')

plt.tight_layout()
plt.show()

In [28]:
df_raw_indexed = df_raw.set_index("row_id")

balance_per_call = (
    df_raw_indexed.loc[df["row_id"], "balance"] /
    df_raw_indexed.loc[df["row_id"], "campaign"].replace(0, np.nan)
)
balance_per_call = balance_per_call.fillna(balance_per_call.median())

contact_intensity = df_raw_indexed.loc[df["row_id"], "campaign"] / (
    df_raw_indexed.loc[df["row_id"], "previous"] + 1
)
contact_intensity = contact_intensity.fillna(contact_intensity.median())

month_numeric = df_raw_indexed.loc[df["row_id"], "month"].map(month_mapper).astype(float)

# align derived features positionally with the working dataframe
df["balance_per_call"] = balance_per_call.to_numpy()
df["contact_intensity_norm"] = contact_intensity.to_numpy()
df["month_sin"] = np.sin(2 * np.pi * month_numeric.to_numpy() / 12)
df["month_cos"] = np.cos(2 * np.pi * month_numeric.to_numpy() / 12)

balance_per_call_scaler = RobustScaler()
df[["balance_per_call"]] = balance_per_call_scaler.fit_transform(df[["balance_per_call"]])

contact_intensity_scaler = MinMaxScaler()
df[["contact_intensity_norm"]] = contact_intensity_scaler.fit_transform(df[["contact_intensity_norm"]])


In [29]:
df.head(20)

,age,education,balance,day,month,campaign,previous,y,row_id,pdays_recency,...,contact_telephone,balance_outlier_flag,campaign_outlier_flag,campaign_unit,previous_unit,pdays_unit,balance_per_call,contact_intensity_norm,month_sin,month_cos
0,-1.06,0.00,1.53,0.60,10.00,0.00,-0.43,0,0,-0.74,...,False,0,0,0.00,-0.50,-0.87,1.93,0.06,-0.87,0.50
1,-0.77,1.00,1.91,0.33,5.00,0.00,2.65,0,1,1.03,...,False,1,0,0.00,0.93,0.36,5.64,0.01,0.50,-0.87
2,-0.58,2.00,1.46,0.50,4.00,0.00,0.90,0,2,0.96,...,False,0,0,0.00,0.68,0.73,1.39,0.03,0.87,-0.50
3,-1.06,2.00,1.48,0.07,6.00,0.92,-0.43,0,3,0.64,...,False,0,0,0.77,-0.36,0.53,0.18,0.25,0.00,-1.00
4,1.69,1.00,1.21,0.13,5.00,0.00,-0.43,0,4,-1.04,...,False,0,0,0.00,-0.38,-0.93,-0.27,0.06,0.50,-0.87
5,-0.58,2.00,1.36,0.73,2.00,0.41,2.22,0,5,-0.63,...,False,0,0,0.17,0.95,-0.27,0.19,0.03,0.87,0.50
6,-0.49,2.00,1.27,0.43,5.00,0.00,1.67,0,6,0.96,...,False,0,0,0.00,0.87,0.50,0.11,0.02,0.50,-0.87
7,-0.21,1.00,1.24,0.17,5.00,0.41,-0.43,0,7,0.77,...,False,0,0,0.42,-0.44,0.80,-0.18,0.12,0.50,-0.87
8,-0.02,2.00,1.25,0.43,5.00,0.41,-0.43,0,8,0.58,...,False,0,0,0.49,-0.51,0.70,-0.14,0.12,0.50,-0.87
9,0.17,0.00,1.19,0.53,4.00,0.00,1.67,0,9,-1.09,...,False,0,0,0.00,0.84,-0.54,-0.38,0.02,0.87,-0.50


## G. Save the Prepared Dataset

In [30]:
df = df.drop(columns=["row_id"])

remaining_missing = int(df.isna().sum().sum())
print(f"Remaining missing values: {remaining_missing}")
print(f"Prepared shape: {df.shape}")

df.to_csv("bank_marketing_prepared.csv", index=False)
print("Exported prepared dataset to bank_marketing_prepared.csv")

Remaining missing values: 0
Prepared shape: (4478, 41)
Exported prepared dataset to bank_marketing_prepared.csv


In [31]:
df.head(20)

,age,education,balance,day,month,campaign,previous,y,pdays_recency,pdays_missing,...,contact_telephone,balance_outlier_flag,campaign_outlier_flag,campaign_unit,previous_unit,pdays_unit,balance_per_call,contact_intensity_norm,month_sin,month_cos
0,-1.06,0.00,1.53,0.60,10.00,0.00,-0.43,0,-0.74,1,...,False,0,0,0.00,-0.50,-0.87,1.93,0.06,-0.87,0.50
1,-0.77,1.00,1.91,0.33,5.00,0.00,2.65,0,1.03,0,...,False,1,0,0.00,0.93,0.36,5.64,0.01,0.50,-0.87
2,-0.58,2.00,1.46,0.50,4.00,0.00,0.90,0,0.96,0,...,False,0,0,0.00,0.68,0.73,1.39,0.03,0.87,-0.50
3,-1.06,2.00,1.48,0.07,6.00,0.92,-0.43,0,0.64,1,...,False,0,0,0.77,-0.36,0.53,0.18,0.25,0.00,-1.00
4,1.69,1.00,1.21,0.13,5.00,0.00,-0.43,0,-1.04,1,...,False,0,0,0.00,-0.38,-0.93,-0.27,0.06,0.50,-0.87
5,-0.58,2.00,1.36,0.73,2.00,0.41,2.22,0,-0.63,0,...,False,0,0,0.17,0.95,-0.27,0.19,0.03,0.87,0.50
6,-0.49,2.00,1.27,0.43,5.00,0.00,1.67,0,0.96,0,...,False,0,0,0.00,0.87,0.50,0.11,0.02,0.50,-0.87
7,-0.21,1.00,1.24,0.17,5.00,0.41,-0.43,0,0.77,1,...,False,0,0,0.42,-0.44,0.80,-0.18,0.12,0.50,-0.87
8,-0.02,2.00,1.25,0.43,5.00,0.41,-0.43,0,0.58,1,...,False,0,0,0.49,-0.51,0.70,-0.14,0.12,0.50,-0.87
9,0.17,0.00,1.19,0.53,4.00,0.00,1.67,0,-1.09,0,...,False,0,0,0.00,0.84,-0.54,-0.38,0.02,0.87,-0.50
